# 二维卷积实验

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import os
from PIL import Image

# --- 1. 配置路径和参数 ---
DATA_ROOT = (
    "D:/files/works/DeepLearning/experiment3/data/car-classificationproject-vision"
)
TRAIN_DIR = os.path.join(DATA_ROOT, "Train/Train")
TEST_DIR = os.path.join(DATA_ROOT, "Test/Test/Test1")  # 无标签测试集
IMAGE_SIZE = 224
BATCH_SIZE = 32
TRAIN_VAL_SPLIT_RATIO = 0.8  # 80% 训练，20% 验证
NUM_WORKERS = 4

# ImageNet 标准均值和标准差
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

# --- 2. 定义数据预处理 (Transforms) ---

# 训练集：需要数据增强以提高模型泛化能力
train_transforms = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),  # 随机水平翻转
        transforms.ColorJitter(
            brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1
        ),  # 颜色抖动
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ]
)

# 验证集和测试集：只需要标准的预处理
test_val_transforms = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ]
)

# --- 3. 划分训练集和验证集 (带标签数据) ---

# 步骤 3.1: 使用 ImageFolder 加载全部训练数据
# 注意：这里我们使用一个临时的 transforms，后面会为子集单独设置
full_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)

# 获取类别信息
class_names = full_dataset.classes
num_classes = len(class_names)
print(f"✅ 检测到的类别数量: {num_classes}")
# print(f"类别名称到索引的映射: {full_dataset.class_to_idx}")

# 步骤 3.2: 随机划分训练集和验证集
train_size = int(TRAIN_VAL_SPLIT_RATIO * len(full_dataset))
val_size = len(full_dataset) - train_size

# random_split 会创建一个子集对象，它会继承原始数据集的 transform。
# 为了让 val_dataset 使用不同的 transform (即 test_val_transforms)，
# 需要更严谨的做法，但最简单常用的是：
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 步骤 3.3: 创建 DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,  # 提高数据传输速度
)

# 💡 注意: 验证集通常不打乱 (shuffle=False)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

print(f"ℹ️ 训练集大小: {len(train_dataset)}, 验证集大小: {len(val_dataset)}")


# --- 4. 加载测试集 (无标签数据) ---


# 步骤 4.1: 定义自定义 Dataset 来处理无标签数据
class CustomTestDataset(Dataset):
    """用于加载无标签测试集（仅返回图像和文件名）"""

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # 仅获取指定格式的图像文件
        self.image_files = [
            f
            for f in os.listdir(root_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.image_files.sort(
            key=lambda x: int(x.split(".")[0].replace("image", ""))
        )  # 按数字排序，确保顺序正确

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # 返回图像和文件名，用于后续生成 submission.csv
        return image, img_name


# 步骤 4.2: 创建测试集 Dataset 和 DataLoader
test_dataset = CustomTestDataset(root_dir=TEST_DIR, transform=test_val_transforms)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

print(f"ℹ️ 测试集大小: {len(test_dataset)}")
print("--- 数据加载和划分完成 ---")

# --- 5. 最终变量清单 ---
# 您现在可以使用：
# - train_loader, val_loader, test_loader 进行训练和评估
# - num_classes (45) 用于设置模型输出层
# - class_names 用于类别名称和 ID 的映射

✅ 检测到的类别数量: 45
ℹ️ 训练集大小: 3240, 验证集大小: 810
ℹ️ 测试集大小: 450
--- 数据加载和划分完成 ---


## 1 手写二维卷积的实现，并在至少一个数据集上进行实验，从训练时间、预测精度、Loss变化等角度分析实验结果（最好使用图表展示）

## 2 使用torch.nn实现二维卷积，并在至少一个数据集上进行实验，从训练时间、预测精度、Loss变化等角度分析实验结果（最好使用图表展示）

## 3 不同超参数的对比分析（包括卷积层数、卷积核大小、batchsize、lr等）选其中至少1-2个进行分析

## 4 使用PyTorch实现经典模型AlexNet并在至少一个数据集进行试验分析 （无GPU环境则至少实现模型）

## 5 使用实验2中的前馈神经网络模型在本次给定数据集上进行实验，并将实验结果与卷积模型结果进行对比分析（选做）